In [217]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

In [218]:
def tt_solve_time_complexity(s: float, r: float, I:float, d: float, num_iterations: float) -> float:
    t1 = s**3 * r**2 * I**2  # prepare local system
    t2 = s**3 * r * I**3  # in-between contractions
    t3 = s**6 * I**6  # direct solve of local system
    t4 = s**3 * I**3  # SVD truncation
    return num_iterations * d * (t1 + t2 + t3 + t4)

In [219]:
def matrix2mpo_time_complexity(z: float, r: float, I:float, d: float) -> float:
    return z + d * I**2 * r**2

def tt_svd_vec_time_complexity(r: float, I:float, n:float) -> float:
    return n * I * r**3

def tt_svd_mat_time_complexity(r: float, I:float, n:float) -> float:
    return n**2 * I**2 * r**3

def tt_sol2vec_sol_time_complexity(s: float, d: float, n: float) -> float:
    return d * n * s**4

In [220]:
def cg_time_complexity(z: float, cond_num: float) -> float:
    return z * np.sqrt(cond_num)

In [221]:
# Setting 1: we can keep s, number of iterations, d are log(n)
def tt_solve_time_complexity_log_assumption(r: float, I:float, n:float) -> float:
    return tt_solve_time_complexity(s=np.log(n), r=r, I=I, d=np.log(n), num_iterations=np.log(n))

def tt_solve_time_complexity_full_log_assumption(n:float) -> float:
    return tt_solve_time_complexity_log_assumption(r=np.log(n), I=np.log(n), n=n)

# assume log(n) except TT-ranks, those are const, s=2
def tt_solve_time_complexity_full_const_s_log_assumption(n:float) -> float:
    return tt_solve_time_complexity(s=2, r=np.log(n), I=np.log(n), d=np.log(n), num_iterations=np.log(n))

# assume log(n) except: s=2, r=2
def tt_solve_time_complexity_full_const_I_s_log_assumption(n:float) -> float:
    return tt_solve_time_complexity(s=2, r=np.log(n), I=2, d=np.log(n), num_iterations=np.log(n))

# assume log(n) except: I=2
def tt_solve_time_complexity_full_const_I_log_assumption(n:float, I:float) -> float:
    return tt_solve_time_complexity(s=np.log(n), r=np.log(n), I=I, d=np.log(n), num_iterations=np.log(n))

In [222]:
def tt_solve_with_conversions_time_complexity(s: float, r:float, I: float, d: float, num_it: float, z:float, n:float) -> float:
    v = tt_solve_time_complexity(s=s, r=r, I=I, d=d, num_iterations=num_it) + matrix2mpo_time_complexity(z=z, r=r, I=I, d=d) + tt_sol2vec_sol_time_complexity(s=s, d=d, n=n) + tt_svd_vec_time_complexity(r=r, I=I, n=n)
    return v

def tt_solve_with_conversions_time_complexity_some_log(s: float, r:float, I: float, n:float) -> float:
    return tt_solve_with_conversions_time_complexity(s=s, r=r, I=I, n=n, d=np.log(n), num_it=np.log(n), z=np.log(n))

In [223]:
xs = np.logspace(1, 20, num=int(1e+3))
xs.shape

(1000,)

In [224]:
tt0_ys = [tt_solve_time_complexity_full_const_I_s_log_assumption(x) for x in xs] 
tt0_label = "$\\text{TT-solve}, O(\log(n)) \\text{, } I=2 \\text{ and } s=2$"

tt1_ys = [tt_solve_time_complexity_full_const_I_log_assumption(x, I=2) for x in xs] 
tt1_label = "$\\text{TT-solve}, O(\log(n)) \\text{, } I=2 \\text{ xor } s=2$"  # they are the same

# tt2_ys = [tt_solve_time_complexity_full_const_s_log_assumption(x) for x in xs]  
# tt2_label = "$\\text{TT-solve}, O(\log(n)), s=2$"

tt3_ys = [tt_solve_time_complexity_full_log_assumption(x) for x in xs]
tt3_label = "$\\text{TT-solve, all } O(\log(n))$"

cg_cond1_ys = [cg_time_complexity(x, cond_num=1e+3) for x in xs]
cg_cond1_label = "$\\text{CG}, z = O(n), \kappa = 10^2$"
cg_cond2_ys = [cg_time_complexity(x, cond_num=10e+10) for x in xs]
cg_cond2_label = "$\\text{CG}, z = O(n), \kappa = 10^{10}$"
cg_cond3_ys = [cg_time_complexity(x, cond_num=10e+20) for x in xs]
cg_cond3_label = "$\\text{CG}, z = O(n), \kappa = 10^{20}$"

In [225]:
df = pd.DataFrame(np.transpose(np.stack([xs, tt0_ys, tt1_ys, tt3_ys, cg_cond1_ys, cg_cond2_ys, cg_cond3_ys])), columns=["x", tt0_label, tt1_label, tt3_label, cg_cond1_label, cg_cond2_label, cg_cond3_label])

In [226]:
melted_df = df.melt(id_vars=["x"])
melted_df.head()

,x,variable,value
0,10.000000,"$\text{TT-solve}, O(\log(n)) \text{, } I=2 \te...",23736.736673
1,10.447660,"$\text{TT-solve}, O(\log(n)) \text{, } I=2 \te...",24699.520073
2,10.915359,"$\text{TT-solve}, O(\log(n)) \text{, } I=2 \te...",25684.042409
3,11.403996,"$\text{TT-solve}, O(\log(n)) \text{, } I=2 \te...",26690.488694
4,11.914507,"$\text{TT-solve}, O(\log(n)) \text{, } I=2 \te...",27719.046758


In [227]:
line_styles = {
    tt0_label: "dot",
    tt1_label: 'dashdot',
    # tt2_label: 'dot',
    tt3_label: 'longdashdot',
    cg_cond1_label: 'solid',
    cg_cond2_label: 'dash',
    cg_cond3_label: 'longdash',
}


In [228]:
fig = px.line(melted_df, x="x", y="value", color="variable", line_dash='variable', 
              line_dash_map=line_styles,
              log_y=True, log_x=True,
                  labels={
                      "x": "Matrix size (n)",
                      "value": "Estimated runtime (FLOPs)",
                      "variable": "Solver, assumptions",
                  })
fig.update_layout(
        title={
            'text': "Runtime comparison of CG and TT-solve without conversions",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
pio.write_image(fig, "plots/tt_solve_without_conversions_vs_cg_runtime_estimates.pdf", scale=1, width=800, height=500)

In [229]:
# vec, mat to TT conversions
tt0_ys = [matrix2mpo_time_complexity(z=x, r=np.log(x), I=np.log(x), d=np.log(x)) for x in xs] 
tt0_label = "$\\text{matrix2mpo, all } O(\log(n))$"

tt1_ys = [tt_svd_mat_time_complexity(r=np.log(x), I=np.log(x), n=x) for x in xs]  
tt1_label = "$\\text{TT-SVD, matrix, all } O(\log(n))$"

tt2_ys = [tt_svd_vec_time_complexity(r=np.log(x), I=np.log(x), n=x) for x in xs] 
tt2_label = "$\\text{TT-SVD, rhs, all } O(\log(n))$"

tt3_ys = [tt_svd_vec_time_complexity(r=2, I=np.log(x), n=x) for x in xs]
tt3_label = "$\\text{TT-SVD, rhs, all } O(\log(n))\\text{, } r=2$"

cg_cond1_ys = [cg_time_complexity(x, cond_num=1e+3) for x in xs]
cg_cond1_label = "$\\text{CG}, z = O(n), \kappa = 10^2$"
# cg_cond2_ys = [cg_time_complexity(x, cond_num=10e+10) for x in xs]
# cg_cond2_label = "$\\text{CG}, z = O(n), \kappa = 10^{10}$"
cg_cond3_ys = [cg_time_complexity(x, cond_num=10e+20) for x in xs]
cg_cond3_label = "$\\text{CG}, z = O(n), \kappa = 10^{20}$"

In [230]:
df = pd.DataFrame(np.transpose(np.stack([xs, tt0_ys, tt1_ys, tt2_ys, tt3_ys, cg_cond1_ys, cg_cond3_ys])), columns=["x", tt0_label, tt1_label, tt2_label, tt3_label, cg_cond1_label, cg_cond3_label])
melted_df = df.melt(id_vars=["x"])
melted_df.head()
line_styles = {
    tt0_label: "dot",
    tt1_label: 'dashdot',
    tt2_label: 'longdashdot',
    tt3_label: 'dash',
    cg_cond1_label: 'solid',
    # cg_cond2_label: 'dash',
    cg_cond3_label: 'longdash',
}

In [231]:
fig = px.line(melted_df, x="x", y="value", color="variable", line_dash='variable', 
              line_dash_map=line_styles,
              log_y=True, log_x=True,
                  labels={
                      "x": "Matrix size (n)",
                      "value": "Estimated runtime (FLOPs)",
                      "variable": "Method, assumptions",
                  })
fig.update_layout(
        title={
            'text': "Runtime comparison of CG and conversions to TT",
            'x': 0.4,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
pio.write_image(fig, "plots/conversion2tt_vs_cg_runtime_estimates.pdf", scale=1, width=800, height=500)

In [232]:
# TT to vec conversions
tt0_ys = [tt_sol2vec_sol_time_complexity(s=np.log(x), d=np.log(x), n=x) for x in xs] 
tt0_label = "$\\text{TT-sol2vector, all } O(\log(n))$"

tt1_ys = [tt_sol2vec_sol_time_complexity(s=2, d=np.log(x), n=x) for x in xs]
tt1_label = "$\\text{TT-sol2vector, } O(\log(n))\\text{, } s=2$"

cg_cond1_ys = [cg_time_complexity(x, cond_num=1e+3) for x in xs]
cg_cond1_label = "$\\text{CG}, z = O(n), \kappa = 10^2$"
cg_cond2_ys = [cg_time_complexity(x, cond_num=10e+10) for x in xs]
cg_cond2_label = "$\\text{CG}, z = O(n), \kappa = 10^{10}$"
cg_cond3_ys = [cg_time_complexity(x, cond_num=10e+20) for x in xs]
cg_cond3_label = "$\\text{CG}, z = O(n), \kappa = 10^{20}$"

In [233]:
df = pd.DataFrame(np.transpose(np.stack([xs, tt0_ys, tt1_ys, cg_cond1_ys, cg_cond2_ys, cg_cond3_ys])), columns=["x", tt0_label, tt1_label, cg_cond1_label, cg_cond2_label, cg_cond3_label])
melted_df = df.melt(id_vars=["x"])
melted_df.head()
line_styles = {
    tt0_label: "dot",
    tt1_label: 'dashdot',
    #tt2_label: 'longdashdot',
    #tt3_label: 'dash',
    cg_cond1_label: 'solid',
    cg_cond2_label: 'dash',
    cg_cond3_label: 'longdash',
}

In [234]:
fig = px.line(melted_df, x="x", y="value", color="variable", line_dash='variable', 
              line_dash_map=line_styles,
              log_y=True, log_x=True,
                  labels={
                      "x": "Matrix size (n)",
                      "value": "Estimated runtime (FLOPs)",
                      "variable": "Method, assumptions",
                  })
fig.update_layout(
        title={
            'text': "Runtime comparison of CG and TT-solution to vector conversions",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
pio.write_image(fig, "plots/tt2vec_conversion_vs_cg_runtime_estimates.pdf", scale=1, width=800, height=500)

In [235]:
# TT-solve with conversions to and from TT-format

tt0_ys = [tt_solve_with_conversions_time_complexity_some_log(s=np.log(x), r=np.log(x), I=np.log(x), n=x) for x in xs] 
tt0_label = "$\\text{TT-solve, all } O(\log(n))$"

tt1_ys = [tt_solve_with_conversions_time_complexity_some_log(s=2, r=np.log(x), I=np.log(x), n=x) for x in xs] 
tt1_label = "$\\text{TT-solve, } O(\log(n))\\text{, } s=2$"

tt2_ys = [tt_solve_with_conversions_time_complexity_some_log(s=np.log(x), r=np.log(x), I=2, n=x) for x in xs]
tt2_label = "$\\text{TT-solve, } O(\log(n))\\text{, } I=2$"

tt3_ys = [tt_solve_with_conversions_time_complexity_some_log(s=2, r=np.log(x), I=2, n=x) for x in xs]
tt3_label = "$\\text{TT-solve, } O(\log(n))\\text{, } I=2\\text{, }s=2$"

cg_cond1_ys = [cg_time_complexity(x, cond_num=1e+3) for x in xs]
cg_cond1_label = "$\\text{CG}, z = O(n), \kappa = 10^2$"

cg_cond3_ys = [cg_time_complexity(x, cond_num=10e+20) for x in xs]
cg_cond3_label = "$\\text{CG}, z = O(n), \kappa = 10^{20}$"

In [236]:
df = pd.DataFrame(np.transpose(np.stack([xs, tt0_ys, tt1_ys, tt2_ys, tt3_ys, cg_cond1_ys, cg_cond3_ys])), columns=["x", tt0_label, tt1_label, tt2_label, tt3_label, cg_cond1_label, cg_cond3_label])
melted_df = df.melt(id_vars=["x"])
melted_df.head()
line_styles = {
    tt0_label: "dot",
    tt1_label: 'dashdot',
    tt2_label: 'longdashdot',
    tt3_label: 'dash',
    cg_cond1_label: 'solid',
    # cg_cond2_label: 'dash',
    cg_cond3_label: 'longdash',
}


In [237]:
fig = px.line(melted_df, x="x", y="value", color="variable", line_dash='variable', 
              line_dash_map=line_styles,
              log_y=True, log_x=True,
                  labels={
                      "x": "Matrix size (n)",
                      "value": "Estimated runtime (FLOPs)",
                      "variable": "Solver, assumptions",
                  })
fig.update_layout(
        title={
            'text': "Runtime comparison of CG and TT-solve with conversions",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
pio.write_image(fig, "plots/tt_solve_with_conversion_vs_cg_runtime_estimates.pdf", scale=1, width=800, height=500)